### Packages Import

In [1]:
import os

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

### Functions

In [14]:
def MHH_build_sows_df_1stStep(demographics_df, encounters_df):
    
    ''' In MHH location column is in the encounters_df'''      # Note to Joey, to move location into encounters_df using left join
    
    '''# Good to know:
    #              discharge_timefrom the Hospital
    #              Timestamp for the Demograpic table = checkin_time coming from the encounters table
    #              checkin_time = arrival to ED    '''

    sows_df = demographics_df.merge(encounters_df,on=['patient_id', 'encounter_id'], how = 'inner')
    
    # Create a Timestamp column that equals the checkin_time column in encounters_df table. 
    sows_df['Timestamp'] = encounters_df.checkin_time            
    
    # Rename column names:
    sows_df.rename(columns = {'checkin_time':'arrival_time', 'admission_time': 'admit_time'}, inplace = True)   
    
    sows_df = sows_df[['patient_id', 'encounter_id', 'age', 'Timestamp', 'arrival_time', 'admit_time', 'discharge_time', 'death_time', 'visit_type', 'location']] 
    
    
    # remove duplicates:
    sows_df = sows_df.drop_duplicates()
    
    print(sows_df.shape)

    return sows_df

### Uploading Data

In [2]:
%env SET0 = bec-data-MetroHealth-Dascena:/HarmonizedT-FeatureT/Notebooks/
    
datadir0 = "data/" + os.environ['SET0'].replace(':','/')
print("Python variable datadir=" + datadir0)

env: SET0=bec-data-MetroHealth-Dascena:/HarmonizedT-FeatureT/Notebooks/
Python variable datadir=data/bec-data-MetroHealth-Dascena//HarmonizedT-FeatureT/Notebooks/


In [3]:
%%bash
DIR=data/${SET0/:/\/}
echo local directory: $DIR
mkdir -p $DIR
dx download -rf "$SET0/Upload_Data.ipynb" --output $DIR    # $SET/* to download all xlsx files
ls -lh $DIR

local directory: data/bec-data-MetroHealth-Dascena//HarmonizedT-FeatureT/Notebooks/
total 28K
-rw-r--r-- 1 root root 26K Sep  7 14:16 Upload_Data.ipynb


In [4]:
%run data/bec-data-MetroHealth-Dascena//HarmonizedT-FeatureT/Notebooks/Upload_Data.ipynb

env: SET=joey-mhh-sandbox:/harmonized_output/
Python variable datadir=data/joey-mhh-sandbox//harmonized_output/
local directory: data/joey-mhh-sandbox//harmonized_output/
total 16G
-rw-r--r-- 1 root root 285M Sep  7 14:17 core
-rw-r--r-- 1 root root  56M Sep  7 14:18 demographics.csv
-rw-r--r-- 1 root root  22M Sep  7 14:18 demographics_2022-05-24.csv
-rw-r--r-- 1 root root  22M Sep  7 14:17 demographics_2022-05-26.csv
-rw-r--r-- 1 root root 998M Sep  7 14:18 diagnoses.csv
-rw-r--r-- 1 root root 998M Sep  7 14:18 diagnoses_2022-05-24.csv
-rw-r--r-- 1 root root 998M Sep  7 14:17 diagnoses_2022-05-26.csv
-rw-r--r-- 1 root root  44M Sep  7 14:18 encounters.csv
-rw-r--r-- 1 root root  45M Sep  7 14:18 encounters_2022-05-24.csv
-rw-r--r-- 1 root root  50M Sep  7 14:17 encounters_2022-05-26.csv
-rw-r--r-- 1 root root  98M Sep  7 14:16 locations_2022-08-31.csv
-rw-r--r-- 1 root root 1.8G Sep  7 14:18 measurements.csv
-rw-r--r-- 1 root root 4.0G Sep  7 14:18 measurements_2022-05-24.csv
-rw-r--

### Read Harmonized Tables CSV to Pandas

In [5]:
%who DataFrame

demographics_df	 df_MHH_features_df	 diagnoses_df	 encounters_df	 infection_ICD_df	 measurements_df	 medications_df	 meds_df_lookup	 procedures_df	 



### Read (Dmitrii) Feature Tables CSV to Pandas
To mimc structure

In [6]:
measur_lst_D = list(df_MHH_features_df.Measurement.unique())
print(len(measur_lst_D))
measur_lst_D

32


['Medication',
 'Age',
 'visit_type',
 'arrival_time',
 'admit_time',
 'discharge_time',
 'death_time',
 'RespRate',
 'SpO2',
 'FiO2',
 'Temp',
 'HR',
 'SysABP',
 'DiasABP',
 'MDW',
 'WBC',
 'Platelets',
 'Lactate',
 'Creatinine',
 'eGFR',
 'aPTT',
 'INR',
 'NeutBands',
 'Microbiology - Urine Culture',
 'Microbiology - Blood Culture',
 'pH',
 'PaO2',
 'PaCO2',
 'Ventilation',
 'ESRD',
 'CKD',
 'Infection']

### Buidling a New Feature Tables - using Dmitrii Feature Table Structure 
Input: Joey's Harmonized CSV tables (pandas DFs)


##### Using exec() Method in Python to Convert a Python string to a Variable Name

## Start BUILD SOWS

### Stage 1: Extract Information (apply no filter)

#### Start with an empty SOWS df:

In [6]:
sows_df = pd.DataFrame() 

# Check if the above created DataFrame
# Is empty or not using the empty property
print('\nIs this an empty DataFrame?\n')
print(sows_df.empty)


Is this an empty DataFrame?

True


#### Assign Coloumns from Harmonized Tables:  

In [8]:
# Demographics:
sows_df['patient_id'] = demographics_df.patient_id
sows_df['encounter_id'] = encounters_df.encounter_id
sows_df['age'] = encounters_df.age      # data type: float64
sows_df['Timestamp'] = encounters_df.checkin_time              # Timestamp for the Demograpic table == checkin_time coming from the encounters table
sows_df['arrival_time'] = encounters_df.checkin_time              # arrival to ED
sows_df['admit_time'] = encounters_df.admission_time              # admit to -> double check this <-
sows_df['discharge_time'] = encounters_df.discharge_time          # from the ED / Hospital
sows_df['death_time'] = demographics_df.death_time  
sows_df['visit_type'] = encounters_df.visit_type  
sows_df['location'] = encounters_df.location

In [15]:
sows_df = MHH_build_sows_df_1stStep(demographics_df, encounters_df)

(188126, 10)


In [16]:
sows_df.head(2)

,patient_id,encounter_id,age,Timestamp,arrival_time,admit_time,discharge_time,death_time,visit_type,location
0,ec689ab9262082c4b40576b011b675c0b5ee395c,c7737554af0689375d21df57536d4192df03c86c,63.131670,2019-07-28 02:20:00+00:00,2019-07-28 02:20:00+00:00,2019-07-28 02:33:00+00:00,2019-07-28 08:16:00+00:00,NaN,routine admission (elective),emergency
1,ec689ab9262082c4b40576b011b675c0b5ee395c,1e85e52155cae96718bfe487cce723ff9d1af15c,63.138371,2019-07-30 13:26:00+00:00,2019-07-30 13:26:00+00:00,2019-07-30 13:33:00+00:00,2019-08-03 16:18:00+00:00,NaN,non-healthcare point of origin,inpatient


In [9]:
print(sows_df.shape)
sows_df.head(2)

(188126, 10)


,patient_id,encounter_id,age,Timestamp,arrival_time,admit_time,discharge_time,death_time,visit_type,location
0,ec689ab9262082c4b40576b011b675c0b5ee395c,c7737554af0689375d21df57536d4192df03c86c,63.131670,2019-07-28 02:20:00+00:00,2019-07-28 02:20:00+00:00,2019-07-28 02:33:00+00:00,2019-07-28 08:16:00+00:00,NaN,routine admission (elective),emergency
1,ec689ab9262082c4b40576b011b675c0b5ee395c,1e85e52155cae96718bfe487cce723ff9d1af15c,63.138371,2019-07-30 13:26:00+00:00,2019-07-30 13:26:00+00:00,2019-07-30 13:33:00+00:00,2019-08-03 16:18:00+00:00,NaN,non-healthcare point of origin,inpatient


In [10]:
# Limiting age
sows_df = sows_df.loc[(sows_df.age < 90.0) & (sows_df.age >= 18.0)]
print(sows_df.shape)

(164524, 10)


### medications_df

In [11]:
print(medications_df.shape)
medications_df.head(2)

(4050976, 13)


,patient_id,encounter_id,med_class,med_name,ingredient,med_route,med_dose,med_dose_unit,med_iv_duration_seconds,infusion_rate,action,order_time,action_time
0,f81a9a563c10c0be6260e0e139f4062f6963bda4,405d2793eb325028c39e13797b1d3d605e6833c6,NaN,abacavir sulfate 300 mg oral tabs,abacavir,oral,300.0,mg,NaN,NaN,given,2021-05-02T13:14:00Z,2021-05-02T14:18:00Z
1,e374f3aa99a5f63dac42a71d1ff1a1491cefbe64,7d13db6d6e3d2ade2377995c31a33b1a3cad2989,NaN,abacavir sulfate 300 mg oral tabs,abacavir,oral,300.0,mg,NaN,NaN,missed,2020-10-21T23:22:00Z,2020-10-22T13:07:00Z


In [12]:
# Left Join to introduce Meds info into sows_df (left_df) using medications_df (right_df):
sows_df_withMeds = sows_df.merge(medications_df[['patient_id', 'encounter_id', 'med_class', 'med_name', 'med_route', 'med_dose', 'med_dose_unit', 'med_iv_duration_seconds']], on=['patient_id', 'encounter_id'], how='left')
sows_df_withMeds = sows_df_withMeds.drop_duplicates()

In [13]:
print(sows_df_withMeds.shape)   # 872707
sows_df_withMeds.head(2)

(819228, 16)


,patient_id,encounter_id,age,Timestamp,arrival_time,admit_time,discharge_time,death_time,visit_type,location,med_class,med_name,med_route,med_dose,med_dose_unit,med_iv_duration_seconds
0,ec689ab9262082c4b40576b011b675c0b5ee395c,c7737554af0689375d21df57536d4192df03c86c,63.13167,2019-07-28 02:20:00+00:00,2019-07-28 02:20:00+00:00,2019-07-28 02:33:00+00:00,2019-07-28 08:16:00+00:00,NaN,routine admission (elective),emergency,NaN,acetaminophen 500 mg oral tabs,oral,500.0,mg,NaN
1,ec689ab9262082c4b40576b011b675c0b5ee395c,c7737554af0689375d21df57536d4192df03c86c,63.13167,2019-07-28 02:20:00+00:00,2019-07-28 02:20:00+00:00,2019-07-28 02:33:00+00:00,2019-07-28 08:16:00+00:00,NaN,routine admission (elective),emergency,NaN,albuterol sulfate (2.5 mg/3ml) 0.083% inhalati...,nebulization,10.0,(mg/ml)%,NaN


In [14]:
# Coloumns that will also be raws under the Measurement column at the "feature" table:
sows_df_withMeds['arrival_time_copy'] = sows_df_withMeds['arrival_time'] # to facilitate arrivale time in a column called Measurmmet as a raw and a column later after melting  
sows_df_withMeds['discharge_time_copy'] = sows_df_withMeds['discharge_time']
sows_df_withMeds['age_copy'] = sows_df_withMeds['age']
sows_df_withMeds['visit_type_copy'] = sows_df_withMeds['visit_type'] 
sows_df_withMeds['Medication'] = sows_df_withMeds['med_name'] 

In [15]:
# Create Measurement and Value columns using the .melt command:

sows_df_m = sows_df_withMeds.melt(id_vars=['patient_id', 'encounter_id', 'Timestamp','arrival_time_copy', 'discharge_time_copy', 
                                  'age_copy', 'visit_type_copy', 'med_class', 'med_name', 'med_route', 'med_dose', 'med_dose_unit', 'med_iv_duration_seconds', 'location'],
                        var_name="Measurement", value_name="Value")
# Rename column names:
sows_df_m.rename(columns = {'arrival_time_copy':'arrival_time', 'discharge_time_copy': 'discharge_time',
                            'age_copy': 'age', 'visit_type_copy': 'visit_type'}, inplace = True)

# Re-order columns:
sows_df_m = sows_df_m[['Measurement', 'Value', 'patient_id', 'encounter_id', 'Timestamp', 'arrival_time',
       'discharge_time', 'age', 'visit_type', 'med_class', 'med_name',
       'med_route', 'med_dose', 'med_dose_unit', 'med_iv_duration_seconds',
       'location']]

#### measurements_df

In [16]:
measurements_df['type'].unique()

array(['lab', 'microbiology', 'vital'], dtype=object)

In [17]:
measurements_df['description'] = measurements_df['description'].str.lower()

##### labs_df_c - for Urine & Blood culture

In [18]:

lab_lst_c = ['c urine', 'c blood']

labs_df_c = measurements_df.loc[measurements_df['description'].isin(lab_lst_c)]

# Chabge so result_value  => Value   ### Done ###
# also transform the resuts to Growth / No Growth  ### DO not do for now - previous featrue table did not facilitated that - raise this quetion in next group meeting ###

labs_df_c.rename(columns = {'description':'Measurement', 'result_value': 'Value', 'result_time': 'Timestamp'}, inplace = True)
labs_df_c.drop(['name', 'collection_time', 'result_value_numeric', 'result_unit', 'order_time', 'loinc_code', 'type'], axis=1, inplace=True)

#labs_df = labs_df[labs_df.Value.notnull()] # Drop rows with None/NaN
labs_df_c = labs_df_c.drop_duplicates()

print(labs_df_c.shape)

# left Join: sows_df_vitals_m  (left_df) using sows_df (right_df): 
labs_df_c_m = labs_df_c[['Measurement', 'Value', 'Timestamp', 'patient_id', 'encounter_id']].merge(sows_df.loc[:, sows_df.columns != 'Timestamp'], 
                                                                                                               on=['patient_id', 'encounter_id'], how='left')
labs_df_c_m = labs_df_c_m.drop_duplicates()
print(labs_df_c_m.shape)
labs_df_c_m.head(1)

(24388, 5)
(24388, 12)


,Measurement,Value,Timestamp,patient_id,encounter_id,age,arrival_time,admit_time,discharge_time,death_time,visit_type,location
0,c urine,"No growth of greater than 1,000 CFU/ml",2020-12-31 20:49:00+00:00,0dff06fcf1b3352c30565b008c50b0b56bc189e3,9de155c5214157219e7e3ca1df596c50f2cccae3,28.487154,2020-12-31 05:42:00+00:00,2020-12-31 05:42:00+00:00,2020-12-31 07:07:00+00:00,NaN,non-healthcare point of origin,outpatient


##### labs_df - all other labs in the list below

In [19]:
# labs_df
lab_lst = ['MDW','WBC','Platelets','Lactate','creatinine (serum)','ESTIMATED GFR (CKD-EPI)','aPTT','INR','NeutBands','pH', 'PaO2', 'PaCO2']

for i in range(len(lab_lst)):
    lab_lst[i] = lab_lst[i].lower()

labs_df = measurements_df.loc[measurements_df['description'].isin(lab_lst)]

labs_df.rename(columns = {'description':'Measurement', 'result_value_numeric': 'Value', 'result_time': 'Timestamp'}, inplace = True)
labs_df.drop(['name', 'collection_time', 'result_value', 'result_unit', 'order_time', 'loinc_code', 'type'], axis=1, inplace=True)

#labs_df = labs_df[labs_df.Value.notnull()] # Drop rows with None/NaN
labs_df = labs_df.drop_duplicates()

print(labs_df.shape)

# left Join: sows_df_vitals_m  (left_df) using sows_df (right_df): 
labs_df_m = labs_df[['Measurement', 'Value', 'Timestamp', 'patient_id', 'encounter_id']].merge(sows_df.loc[:, sows_df.columns != 'Timestamp'], 
                                                                                                               on=['patient_id', 'encounter_id'], how='left')
labs_df_m = labs_df_m.drop_duplicates()
print(labs_df_m.shape)
labs_df_m.head(1)

(718021, 5)
(718021, 12)


,Measurement,Value,Timestamp,patient_id,encounter_id,age,arrival_time,admit_time,discharge_time,death_time,visit_type,location
0,platelets,236.0,2020-06-14 17:32:00+00:00,214968d3164ad930506f80499096c70e3e91876e,e63c3ec6c30bf4970874402bf40ff78d1577b800,22.270671,2020-06-14 15:19:00+00:00,2020-06-14 15:19:00+00:00,2020-06-16 17:00:00+00:00,NaN,non-healthcare point of origin,inpatient


#### Vital Signs 
Measurement = name, Timestamp = result_time

In [20]:
sows_df_vitals = measurements_df.loc[measurements_df['type']=='vital']
sows_df_vitals_m = sows_df_vitals.loc[sows_df_vitals['name'].isin(['HR', 'Temp', 'RespRate', 'SysABP', 'DiasABP', 'MAP', 'SpO2', 'FiO2'])]
sows_df_vitals_m.rename(columns = {'name':'Measurement', 'result_value_numeric': 'Value', 'result_time': 'Timestamp'}, inplace = True)
sows_df_vitals_m.drop(['description', 'collection_time', 'result_value', 'result_unit', 'order_time', 'loinc_code', 'type'], axis=1, inplace=True)

sows_df_vitals_m = sows_df_vitals_m[sows_df_vitals_m.Value.notnull()] # Drop rows with None/NaN
sows_df_vitals_m = sows_df_vitals_m.drop_duplicates()

In [21]:
print(sows_df_vitals_m.shape)
sows_df_vitals_m.head(1)

(11764126, 5)


,patient_id,encounter_id,Measurement,Value,Timestamp
6462330,0dff06fcf1b3352c30565b008c50b0b56bc189e3,9de155c5214157219e7e3ca1df596c50f2cccae3,HR,92.0,2020-12-31T05:49:00Z


In [22]:
# left Join: sows_df_vitals_m  (left_df) using sows_df (right_df): 
sows_df_vitals_m = sows_df_vitals_m[['Measurement', 'Value', 'Timestamp', 'patient_id', 'encounter_id']].merge(sows_df.loc[:, sows_df.columns != 'Timestamp'], 
                                                                                                               on=['patient_id', 'encounter_id'], how='left')
sows_df_vitals_m = sows_df_vitals_m.drop_duplicates()
print(sows_df_vitals_m.shape)
sows_df_vitals_m.head(1)

(11764126, 12)


,Measurement,Value,Timestamp,patient_id,encounter_id,age,arrival_time,admit_time,discharge_time,death_time,visit_type,location
0,HR,92.0,2020-12-31T05:49:00Z,0dff06fcf1b3352c30565b008c50b0b56bc189e3,9de155c5214157219e7e3ca1df596c50f2cccae3,28.487154,2020-12-31 05:42:00+00:00,2020-12-31 05:42:00+00:00,2020-12-31 07:07:00+00:00,NaN,non-healthcare point of origin,outpatient


In [23]:
set(sows_df_vitals_m.Measurement.unique()) == set(['HR', 'Temp', 'RespRate', 'SysABP', 'DiasABP', 'MAP', 'SpO2', 'FiO2'])

True

#### Same code but lowcase before ... NO NEED

for i in range(len(lab_lst)):
    lab_lst[i] = lab_lst[i].lower()

labs_df = measurements_df.loc[measurements_df['description'].isin(lab_lst)]
print(labs_df.shape) # 31699
labs_df.head(1)

sows_df_vitals_m = sows_df_vitals.loc[sows_df_vitals['name'].isin(['HR', 'Temp', 'RespRate', 'SysABP', 'DiasABP', 'MAP', 'SpO2', 'FiO2'])]
sows_df_vitals_m.rename(columns = {'name':'Measurement', 'result_value_numeric': 'Value', 'result_time': 'Timestamp'}, inplace = True)
sows_df_vitals_m.drop(['description', 'collection_time', 'result_value', 'result_unit', 'order_time', 'loinc_code', 'type'], axis=1, inplace=True)

sows_df_vitals_m = sows_df_vitals_m[sows_df_vitals_m.Value.notnull()] # Drop rows with None/NaN
sows_df_vitals_m = sows_df_vitals_m.drop_duplicates()

sows_df_vitals_m.head()

# left Join: sows_df_vitals_m  (left_df) using sows_df (right_df): 
sows_df_vitals_m = sows_df_vitals_m[['Measurement', 'Value', 'Timestamp', 'patient_id', 'encounter_id']].merge(sows_df.loc[:, sows_df.columns != 'Timestamp'], 
                                                                                                               on=['patient_id', 'encounter_id'], how='left')
sows_df_vitals_m = sows_df_vitals_m.drop_duplicates()
print(sows_df_vitals_m.shape)
sows_df_vitals_m.head(1)

### df_diagnoses

##### Filter for Current Diagnoses
is_history column == True, namely for the SOWS model we want to capture current diagnosesonly.  

is_primary column == True or False 
(True means it is the primary diagnosis, False mean it's a non-primary diagnosis but still current diagnosis fom this encounter) 

In [24]:
diagnoses_df = diagnoses_df[diagnoses_df['is_history'] == False]
print(diagnoses_df.shape)

(45469, 6)


In [25]:
#diagnoses_df_intermediate = pd.read_csv(datadir3 + "/diagnoses_df_intermediate.csv")
diagnoses_df.drop(['is_primary', 'is_history'], axis=1, inplace=True)
diagnoses_df.rename(columns = {'name':'Measurement', 'icd10_code': 'Value'}, inplace = True)

In [26]:
print(diagnoses_df.shape)
diagnoses_df.head(3)

(45469, 4)


,patient_id,encounter_id,Measurement,Value
0,ec689ab9262082c4b40576b011b675c0b5ee395c,1e85e52155cae96718bfe487cce723ff9d1af15c,Infection,B34.8
1,223680eead1f625ee73894eb7ed19b80b96f94ce,20923b528513c6188894bc3ee70ea8f299363881,Infection,J02.0
2,223680eead1f625ee73894eb7ed19b80b96f94ce,09d397b1895e5559a42f56d4c440b5250a80d563,Infection,B37.3


In [27]:
# Left Join to introduce Meds info into diagnoses_df  (left_df) using sows_df (right_df):
sows_df_diagnoses_m = diagnoses_df[['Measurement', 'Value', 'patient_id', 'encounter_id']].merge(sows_df, on=['patient_id', 'encounter_id'], how='left')
sows_df_diagnoses_m = sows_df_diagnoses_m.drop_duplicates()
print(sows_df_diagnoses_m.shape)
sows_df_diagnoses_m.head(1)

(44540, 12)


,Measurement,Value,patient_id,encounter_id,age,Timestamp,arrival_time,admit_time,discharge_time,death_time,visit_type,location
0,Infection,B34.8,ec689ab9262082c4b40576b011b675c0b5ee395c,1e85e52155cae96718bfe487cce723ff9d1af15c,63.138371,2019-07-30 13:26:00+00:00,2019-07-30 13:26:00+00:00,2019-07-30 13:33:00+00:00,2019-08-03 16:18:00+00:00,NaN,non-healthcare point of origin,inpatient


In [28]:
sows_df.head(1)

,patient_id,encounter_id,age,Timestamp,arrival_time,admit_time,discharge_time,death_time,visit_type,location
0,ec689ab9262082c4b40576b011b675c0b5ee395c,c7737554af0689375d21df57536d4192df03c86c,63.13167,2019-07-28 02:20:00+00:00,2019-07-28 02:20:00+00:00,2019-07-28 02:33:00+00:00,2019-07-28 08:16:00+00:00,NaN,routine admission (elective),emergency


#### Procedures_df

In [29]:
procedures_df.head(2)

,Timestamp,start_time,stop_time,cpt_code,proc_name,proc_cat_name,patient_id,encounter_id,proc_name_first,ventilation
0,2016-03-07 05:00:00+00:00,NaN,NaN,8100002000,hc periop supply,hc optime charge-ables,0dff06fcf1b3352c30565b008c50b0b56bc189e3,9de155c5214157219e7e3ca1df596c50f2cccae3,hc,0
1,2016-03-07 05:00:00+00:00,NaN,NaN,59514,hc cesarean delivery only,hc l&d charge-ables,0dff06fcf1b3352c30565b008c50b0b56bc189e3,9de155c5214157219e7e3ca1df596c50f2cccae3,hc,0


In [30]:
# Create Measurement and Value columns using the .melt command:

procedures_df_m = procedures_df.melt(id_vars=['patient_id', 'encounter_id', 'Timestamp', 'start_time', 'stop_time', 'cpt_code', 'proc_name', 'proc_cat_name', 'proc_name_first'],
                                      var_name="Measurement", value_name="Value")

procedures_df_m = procedures_df_m[['Measurement', 'Value', 'Timestamp', 'patient_id', 'encounter_id']]
procedures_df_m = procedures_df_m.drop_duplicates()   # Before: 1966469, after: 341305

In [31]:
print(procedures_df_m.shape)
procedures_df_m.head(2)

(341305, 5)


,Measurement,Value,Timestamp,patient_id,encounter_id
0,ventilation,0,2016-03-07 05:00:00+00:00,0dff06fcf1b3352c30565b008c50b0b56bc189e3,9de155c5214157219e7e3ca1df596c50f2cccae3
6,ventilation,0,2016-03-11 05:00:00+00:00,0dff06fcf1b3352c30565b008c50b0b56bc189e3,9de155c5214157219e7e3ca1df596c50f2cccae3


In [32]:
# left Join: procedures_df_m  (left_df) using sows_df (right_df): 
procedures_df_m = procedures_df_m.merge(sows_df.loc[:, sows_df.columns != 'Timestamp'], on=['patient_id', 'encounter_id'], how='left')
procedures_df_m = procedures_df_m.drop_duplicates()
print(procedures_df_m.shape)
procedures_df_m.head(1)

(341305, 12)


,Measurement,Value,Timestamp,patient_id,encounter_id,age,arrival_time,admit_time,discharge_time,death_time,visit_type,location
0,ventilation,0,2016-03-07 05:00:00+00:00,0dff06fcf1b3352c30565b008c50b0b56bc189e3,9de155c5214157219e7e3ca1df596c50f2cccae3,28.487154,2020-12-31 05:42:00+00:00,2020-12-31 05:42:00+00:00,2020-12-31 07:07:00+00:00,NaN,non-healthcare point of origin,outpatient


In [33]:
sows_df.head(1)

,patient_id,encounter_id,age,Timestamp,arrival_time,admit_time,discharge_time,death_time,visit_type,location
0,ec689ab9262082c4b40576b011b675c0b5ee395c,c7737554af0689375d21df57536d4192df03c86c,63.13167,2019-07-28 02:20:00+00:00,2019-07-28 02:20:00+00:00,2019-07-28 02:33:00+00:00,2019-07-28 08:16:00+00:00,NaN,routine admission (elective),emergency


# ---------------------------------------------------------------------------------------

In [34]:
%who DataFrame

demographics_df	 df_MHH_features_df	 diagnoses_df	 encounters_df	 infection_ICD_df	 labs_df	 labs_df_c	 labs_df_c_m	 labs_df_m	 
measurements_df	 medications_df	 meds_df_lookup	 procedures_df	 procedures_df_m	 sows_df	 sows_df_diagnoses_m	 sows_df_m	 sows_df_vitals	 
sows_df_vitals_m	 sows_df_withMeds	 


## do in one go when you have all dataframes ready 

In [35]:
dfs = [sows_df_m, sows_df_vitals_m, labs_df_c_m, labs_df_m, procedures_df_m, sows_df_diagnoses_m]  

extract_df = pd.concat(dfs, ignore_index=True)

In [36]:
print(extract_df.shape)

(18626976, 18)


In [37]:
len(extract_df.Measurement.unique())

33

In [55]:
extract_df.Measurement.unique()

array(['age', 'arrival_time', 'admit_time', 'discharge_time',
       'death_time', 'visit_type', 'Medication', 'HR', 'SpO2', 'Temp',
       'RespRate', 'MAP', 'FiO2', 'SysABP', 'DiasABP', 'c urine',
       'c blood', 'platelets', 'wbc', 'inr', 'estimated gfr (ckd-epi)',
       'aptt', 'paco2', 'pao2', 'neutbands', 'creatinine (serum)', 'ph',
       'mdw', 'lactate', 'ventilation', 'Infection', 'CKD', 'ESRD'],
      dtype=object)

## Define the cohorts

* We are looking to include only adults under 90 years of age
* We have a list of encounters to exclude (because they were adjudicated)
* Data is NOT splite 80%-20%

### The Lookup table for N=600 Dascena (N = 659 including repeates)

In [46]:
%env SET2 = bec-data-MetroHealth-Dascena:/raw/QC-Preliminary_Investigation
    
datadir2 = "data/" + os.environ['SET2'].replace(':','/')
print("Python variable datadir=" + datadir2)

env: SET2=bec-data-MetroHealth-Dascena:/raw/QC-Preliminary_Investigation
Python variable datadir=data/bec-data-MetroHealth-Dascena//raw/QC-Preliminary_Investigation


In [47]:
%%bash
DIR=data/${SET2/:/\/}
echo local directory: $DIR
mkdir -p $DIR
dx download -rf "$SET2/Subject_to_Encounter_IDmap.csv" --output $DIR
ls -lh $DIR

local directory: data/bec-data-MetroHealth-Dascena//raw/QC-Preliminary_Investigation
total 32K
-rw-r--r-- 1 root root 30K Aug 31 15:56 Subject_to_Encounter_IDmap.csv


### Read Lookup table

In [48]:
Lookup_table = pd.read_csv(datadir2 + "/Subject_to_Encounter_IDmap.csv")
Lookup_table.rename(columns = {'Subject ID Number': 'Sub_ID', 'Encounter ID': 'Encounter'}, inplace = True)

### Exlude Adjudication Data

In [60]:
list_enc_adj_exlude = list(Lookup_table.Encounter.unique())

In [62]:
feature_df = extract_df.loc[~extract_df['encounter_id'].isin(list_enc_adj_exlude)]
feature_df = feature_df.drop_duplicates()

In [65]:
feature_df.head()

,Measurement,Timestamp,Value,admit_time,age,arrival_time,death_time,discharge_time,encounter_id,location,med_class,med_dose,med_dose_unit,med_iv_duration_seconds,med_name,med_route,patient_id,visit_type
0,age,2019-07-28 02:20:00+00:00,63.1317,NaN,63.13167,2019-07-28 02:20:00+00:00,NaN,2019-07-28 08:16:00+00:00,c7737554af0689375d21df57536d4192df03c86c,emergency,NaN,500.0,mg,NaN,acetaminophen 500 mg oral tabs,oral,ec689ab9262082c4b40576b011b675c0b5ee395c,routine admission (elective)
1,age,2019-07-28 02:20:00+00:00,63.1317,NaN,63.13167,2019-07-28 02:20:00+00:00,NaN,2019-07-28 08:16:00+00:00,c7737554af0689375d21df57536d4192df03c86c,emergency,NaN,10.0,(mg/ml)%,NaN,albuterol sulfate (2.5 mg/3ml) 0.083% inhalati...,nebulization,ec689ab9262082c4b40576b011b675c0b5ee395c,routine admission (elective)
2,age,2019-07-28 02:20:00+00:00,63.1317,NaN,63.13167,2019-07-28 02:20:00+00:00,NaN,2019-07-28 08:16:00+00:00,c7737554af0689375d21df57536d4192df03c86c,emergency,NaN,20.0,mg/ml,NaN,guaifenesin 100 mg/5ml oral soln,oral,ec689ab9262082c4b40576b011b675c0b5ee395c,routine admission (elective)
3,age,2019-07-28 02:20:00+00:00,63.1317,NaN,63.13167,2019-07-28 02:20:00+00:00,NaN,2019-07-28 08:16:00+00:00,c7737554af0689375d21df57536d4192df03c86c,emergency,NaN,0.5,-()mg/ml,NaN,ipratropium-albuterol 0.5-2.5 (3) mg/3ml inhal...,nebulization,ec689ab9262082c4b40576b011b675c0b5ee395c,routine admission (elective)
4,age,2019-07-28 02:20:00+00:00,63.1317,NaN,63.13167,2019-07-28 02:20:00+00:00,NaN,2019-07-28 08:16:00+00:00,c7737554af0689375d21df57536d4192df03c86c,emergency,NaN,125.0,mg,NaN,methylprednisolone sodium succ 125 mg inj solr,iv/im,ec689ab9262082c4b40576b011b675c0b5ee395c,routine admission (elective)


In [63]:
feature_df.shape

(17991991, 18)

In [73]:
feature_df.to_csv('feature_df_08312022.csv') 

In [74]:
%%bash
dx upload 'feature_df_08312022.csv' --path /HarmonizedT-FeatureT/Results/ --brief

file-GG7jgJQ0k901QkXqJk22GkpV


In [64]:
feature_df.admit_time.unique()

array([nan, '2020-12-31 05:42:00+00:00', '2020-06-14 15:19:00+00:00', ...,
       '2021-08-07 00:53:00+00:00', '2021-01-16 04:49:00+00:00',
       '2021-02-06 10:39:00+00:00'], dtype=object)

In [70]:
feature_df.loc[feature_df.Measurement == 'HR'].head(1)

,Measurement,Timestamp,Value,admit_time,age,arrival_time,death_time,discharge_time,encounter_id,location,med_class,med_dose,med_dose_unit,med_iv_duration_seconds,med_name,med_route,patient_id,visit_type
5734596,HR,2020-12-31T05:49:00Z,92,2020-12-31 05:42:00+00:00,28.487154,2020-12-31 05:42:00+00:00,NaN,2020-12-31 07:07:00+00:00,9de155c5214157219e7e3ca1df596c50f2cccae3,outpatient,NaN,NaN,NaN,NaN,NaN,NaN,0dff06fcf1b3352c30565b008c50b0b56bc189e3,non-healthcare point of origin


In [71]:
feature_df.loc[feature_df.Measurement == 'age'].head(1)

,Measurement,Timestamp,Value,admit_time,age,arrival_time,death_time,discharge_time,encounter_id,location,med_class,med_dose,med_dose_unit,med_iv_duration_seconds,med_name,med_route,patient_id,visit_type
0,age,2019-07-28 02:20:00+00:00,63.1317,NaN,63.13167,2019-07-28 02:20:00+00:00,NaN,2019-07-28 08:16:00+00:00,c7737554af0689375d21df57536d4192df03c86c,emergency,NaN,500.0,mg,NaN,acetaminophen 500 mg oral tabs,oral,ec689ab9262082c4b40576b011b675c0b5ee395c,routine admission (elective)


In [72]:
df_MHH_features_df.loc[df_MHH_features_df.Measurement == 'Age'].head(1)

,Measurement,Value,Patient_ID,Timestamp,Encounter,admission_time,discharge_time,age,visit_type,med_class,med_name,med_route,med_dose,med_dose_unit,med_iv_duration_seconds,location
158707,Age,54.5428193499622,7189fc9fe0db66d9c102cf817668d3206dd7d3c5,2020-04-17T15:40:00.000Z,a0feefc48a7b61591a447e6f47daebecbcef193c,2020-04-17T15:56:00.000Z,2020-04-17T20:06:00.000Z,54.542819,emergency,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df_MHH_features_df.loc[df_MHH_features_df.Measurement == 'Lactate'].head()

,Measurement,Value,Patient_ID,Timestamp,Encounter,admission_time,discharge_time,age,visit_type,med_class,med_name,med_route,med_dose,med_dose_unit,med_iv_duration_seconds,location
10732554,Lactate,3.3,17d999b08a7075d200ec78360d436ed43b9ccc78,2021-04-29T02:28:00.000Z,0035d01d48bf23128e7cecffa31533c90e4f1fce,2021-04-29T02:26:00.000Z,2021-04-29T05:25:00.000Z,71.665010,emergency,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10732555,Lactate,1.4,051e90d4efb481571c0774b9204e2555836d9473,2020-10-03T12:47:00.000Z,003c884b88b4f139854dca12e3a8ec64f1f05d85,2020-10-03T12:36:00.000Z,2020-10-03T15:05:00.000Z,58.955944,emergency,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10732556,Lactate,2.9,0e6d9a0e00ee53a04545aa6f8e71785267182d53,2021-07-30T18:51:00.000Z,004c4fc9ce8665a81ec609b243d2d1968e285c41,2021-07-30T18:00:00.000Z,2021-07-31T01:58:00.000Z,36.962017,emergency,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10732557,Lactate,1.6,0e6d9a0e00ee53a04545aa6f8e71785267182d53,2021-07-31T00:13:00.000Z,004c4fc9ce8665a81ec609b243d2d1968e285c41,2021-07-30T18:00:00.000Z,2021-07-31T01:58:00.000Z,36.962017,emergency,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10732558,Lactate,6.3,34c75d921e84de20c997b2f1bc49e2752f4f733c,2021-12-18T23:00:00.000Z,007a0d771180b216716fdb6164176081c9cecb28,2021-12-18T22:36:00.000Z,2021-12-31T19:16:00.000Z,82.097190,inpatient,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
measurements_df.loc[measurements_df.name == 'Lactate'].head(1)

,patient_id,encounter_id,name,description,loinc_code,type,result_value,result_value_numeric,result_unit,order_time,collection_time,result_time
215919,b46a3a687120086056515177af4053856297e79d,c0a66361070b9f98c72dc2d7a258c72ba7e794a0,Lactate,cr lact,32693-4,lab,5.5,5.5,mmol/l,2020-01-31 10:54:58+00:00,2020-01-31 10:55:00+00:00,2020-01-31 10:55:00+00:00


In [51]:
lact_lst = measurements_df.name.str.contains('Lactate', regex=False, case=False)
lact_lst.unique

<bound method Series.unique of 0           False
1           False
2           False
3           False
4           False
            ...  
23305122    False
23305123    False
23305124    False
23305125    False
23305126    False
Name: name, Length: 23305127, dtype: object>

In [52]:
measurements_df.head(1)

,patient_id,encounter_id,name,description,loinc_code,type,result_value,result_value_numeric,result_unit,order_time,collection_time,result_time
0,0dff06fcf1b3352c30565b008c50b0b56bc189e3,9de155c5214157219e7e3ca1df596c50f2cccae3,Urine Culture,c urine,630-4,lab,"No growth of greater than 1,000 CFU/ml",NaN,NaN,2020-12-31 06:08:27+00:00,2020-12-31 05:56:00+00:00,2020-12-31 20:49:00+00:00


In [53]:
sum(list(lact_lst))

nan

In [54]:
lact_lst

0           False
1           False
2           False
3           False
4           False
            ...  
23305122    False
23305123    False
23305124    False
23305125    False
23305126    False
Name: name, Length: 23305127, dtype: object